In [7]:
import pandas as pd
df = pd.read_csv('data/statements.csv', encoding='utf8')

In [8]:
statements = df['statements'].values

In [9]:
statements[:5]

array(['Fashion week in NYC is in February ',
       'Roosevelt Field  site did Lindbergh begin his flight from in 1927',
       'Saudi Arabia  country is the holy city of Mecca located in',
       'the sun is  5778.0 hot',
       'banana  flavor filling did the original Twinkies have'],
      dtype=object)

# test spacy Entity Recogmition

In [10]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_lg')

In [11]:
def get_spacy_entities(text, count=3):
    nlp_doc = en_nlp(text)
    return[ent.text for ent in nlp_doc.ents[:count]]

get_spacy_entities(statements[1])

['Roosevelt Field', 'Lindbergh', '1927']

# Test Wikification

In [12]:
def get_wiki_entities(text, count=3):
    import tagme
    from operator import itemgetter
    tagme.GCUBE_TOKEN = "c294fe63-88fa-40e1-b1d4-34b70c887b29-843339462"
    newlist = [{'score': item.score,'mention':item.mention,'entity_title':item.entity_title} for item in tagme.annotate(text).get_annotations(0.1)]
    newlist = sorted(newlist, key=itemgetter('score'), reverse=True)
    return [{'mention':item['mention'],'entity_title':item['entity_title']} for item in newlist[:count]]

get_wiki_entities(statements[1])

[{'entity_title': 'Roosevelt Field (airport)', 'mention': 'Roosevelt Field'},
 {'entity_title': 'Charles Lindbergh', 'mention': 'Lindbergh'},
 {'entity_title': 'Flight International', 'mention': 'flight'}]

# semantic role labeling
allennlp # http://demo.allennlp.org/semantic-role-labeling, sorces doesnot work, need to to try docker
nlpnet #only portoguese
senna = python(pntl == practnlptools) cant use becouse of dependensies of senna and standford parser 

# Rake keyword extraction

In [13]:
from rake_nltk import Rake

def get_rake_entities(text, count=3):
    r = Rake() 
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[:count] # To get keyword phrases ranked highest to lowest.
    
get_rake_entities(statements[1])

['roosevelt field site', 'lindbergh begin', 'flight']

# Plan

# Search in Wiipedia

In [18]:
def get_merget_entities(text):
    entities = get_wiki_entities(text)
    if len(entities)<2:
        spacy_entities =  get_spacy_entities(text)
        for spacy_entity in spacy_entities:
            for entity in entities:
                if (spacy_entity not in entity['mention']) and (entity['mention'] not in spacy_entity):
                    entities.append({'mention':spacy_entity,'entity_title':spacy_entity})
    return entities

In [19]:
import wikipedia
wikipedia.set_lang("en")
# get data from wikipedia https://pypi.python.org/pypi/wikipedia
def get_wiki_content(entity):
    fined_page_names = wikipedia.search(entity)
    if len(fined_page_names)>0:
        try:
            wiki_page = wikipedia.page(fined_page_names[0])
            return wiki_page.content
        except:
            return ''
    else:
        return ''
    
get_wiki_content('Roosevelt Field (airport)')

'Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York. Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.\nIn 1919, it was renamed in honor of President Theodore Roosevelt\'s son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh\'s 1927 solo transatlantic flight. It was also used by other pioneering aviators, including Amelia Earhart and Wiley Post.\n\n\n== History ==\nThe Hempstead Plains Aerodrome originally encompassed 900 to 1,000 acres (405 ha) east of and abutting Clinton Road, south of and adjacent to Old Country Road, and west of Merrick Avenue. A bluff 15 feet in elevation divided the plain into two large fields. The U.

In [22]:
from nltk.tokenize import sent_tokenize
def text_to_sents(text):
    sents = sent_tokenize(text)
    #lematized_sents =[]
    #for sent in sents:
    #    doc = en_nlp(sent)
    #    lematized_sent=' '.join([token.lemma_ for token in doc])
    #    lematized_sents.append(lematized_sent)
    #return lematized_sents
    return sents
text_to_sents("Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York. Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.\nIn 1919, it was renamed in honor of President Theodore Roosevelt\'s son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh\'s 1927 solo transatlantic flight. ")

['Roosevelt Field is a former airport, located 2.3 miles (3.7 km) east-southeast of Mineola, Long Island, New York.',
 'Originally called the Hempstead Plains Aerodrome, or sometimes Hempstead Plains field or the Garden City Aerodrome, it was a training field (Hazelhurst Field) for the Air Service, United States Army during World War I.',
 "In 1919, it was renamed in honor of President Theodore Roosevelt's son, Quentin, who was killed in air combat during World War I.\nRoosevelt Field was the takeoff point for many historic flights in the early history of aviation, including Charles Lindbergh's 1927 solo transatlantic flight."]

In [23]:
def get_comention_score_in_sents(entity, sents):
    counter=0
    for sent in  sents:
        if entity in sent:
            counter=counter+1
    return counter

In [24]:
def check_true(text):
    true_counter=0
    entities = get_merget_entities(text)
    for entity in entities:
        wiki_content = get_wiki_content(entity['entity_title'])
        wiki_sents = text_to_sents(wiki_content)
        for entity_2 in entities:
            if entity_2['mention']!=entity['mention']:
                true_counter=true_counter+ get_comention_score_in_sents(entity_2['mention'],wiki_sents)
                
    return true_counter

In [25]:
check_true(statements[1])

53

In [26]:
from tqdm import tqdm
coefs = []
for i in tqdm(range(df.shape[0])):
    coefs.append(check_true(statements[i]))

  2%|▏         | 21/867 [02:01<1:21:22,  5.77s/it]/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|██████████| 867/867 [1:20:36<00:00,  5.58s/it]


In [27]:
df['coefs']=coefs

In [ ]:
df.to_csv('data/statements.csv', encoding='utf8', index=False)

# Result by searchink key words in Wiki abstract


In [40]:
df[:10][['statements','coefs']]

,statements,coefs
0,Fashion week in NYC is in February,0
1,Roosevelt Field site did Lindbergh begin his ...,53
2,Saudi Arabia country is the holy city of Mecc...,36
3,the sun is 5778.0 hot,0
4,banana flavor filling did the original Twinki...,14
5,Home on the Range is the state song of Kansas,129
6,Crimean war is connected with the book `` Cha...,10
7,the first atomic bomb detonated in New Mexico,45
8,Europe continent is Scotland in,261
9,Puerto Rico island did the U.S. gain after th...,224


In [38]:
correct_answer_count = df[df['coefs']>0].shape[0]/df.shape[0]
correct_answer_count

0.8708189158016147

# Lets try to increase this value by using sparql and query DBPedia

In [41]:
df[df['coefs']==0]

,id,type,question,answer,answer_value,statements,coefs
0,1756,factoid,When is Fashion week in NYC?,Sept?(ember)?|Feb(ruary)?,February,Fashion week in NYC is in February,0
3,1992,factoid,How hot is the sun?,"(5,?778|6,?[05]00)\s*K|(5,?505|6,?[50]00).*C|(...",5778.0,the sun is 5778.0 hot,0
11,1663,factoid,What are the people who make fireworks called?,Pyrotechnists,Pyrotechnists,Pyrotechnists make fireworks called,0
15,2364,factoid,How hot does it get in Death Valley?,\b134\b|56\.7|\b116\b|\b47\b|56\.6\°C,56.6°C,it get in Death Valley does 56.6°C hot,0
24,2094,factoid,When was JFK born?,1917,1917,JFK born in 1917,0
35,2186,factoid,What are the measurements for a king-size bed?,"76\s*(inch|"").*80\s*(inch|"")",76 s inch 80 s inch,76 s inch 80 s inch are the measurem...,0
61,1582,factoid,How high is a hand?,4\s*in(ches)?,4 in,a hand is 4 in high,0
70,1667,factoid,What is the abbreviation for the London stock ...,L.?S.?E.?,"LuSBE,","LuSBE, is the abbreviation for the London stoc...",0
75,1549,factoid,What year was the IBM PC invented?,1981,1981,1981 year was the IBM PC invented,0
76,2333,factoid,How old do you have to be to join the U.S. Navy?,\b17\b,17,do you have to be to join the U.S. Navy 17 old,0
